# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [101]:
Summaries_file = 'data/emotion_Summaries.pkl.bz2'
Abstracts_file = 'data/emotion_Abstracts.pkl.bz2'

In [102]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [103]:
Summaries[34648542]

paper(title='Electrophysiological correlates of interference control in the modified emotional Stroop task with emotional stimuli differing in valence, arousal, and subjective significance.', authors=['Imbir KK', 'Pastwa M', 'Duda-Goławska J', 'Sobieszek A', 'Jankowska M', 'Modzelewska A', 'Wielgopolan A', 'Żygierewicz J'], year=2021, doi='10.1371/journal.pone.0258177')

In [104]:
Abstracts[34648542]

'The role of emotional factors in maintaining cognitive control is one of the most intriguing issues in understanding emotion-cognition interactions. In the current experiment, we assessed the role of emotional factors (valence, arousal, and subjective significance) in perceptual and conceptual inhibition processes. We operationalised both processes with the classical cognitive paradigms, i.e., the flanker task and the emotional Stroop task merged into a single experimental procedure. The procedure was based on the presentation of emotional words displayed in four different font colours flanked by the same emotional word printed with the same or different font colour. We expected to find distinct effects of both types of interference: earlier for perceptual and later for emotional interference. We also predicted an increased arousal level to disturb inhibitory control effectiveness, while increasing the subjective significance level should improve this process. As we used orthogonal ma

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [105]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [106]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(34648542)
display_summary(34648542, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [107]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a few seconds:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

Let's see what's in the index for the example term 'amsterdam':

In [108]:
print(inverted_index['amsterdam'])

[9756244, 16916567, 21859206, 25186285, 26784347, 29218587, 29406610, 33741990]


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'utrecht':

In [109]:
query_word = 'utrecht'
for i in inverted_index[query_word]:
    display_summary(i, show_abstract=True)

----------

# Tasks

**Your name:** Nikitas Filosofof

In [110]:
print(inverted_index['utrecht'],inverted_index['amsterdam'])

min((inverted_index['utrecht'],inverted_index['amsterdam']))

[9309949, 12887631, 22419565, 31854264, 34512487] [9756244, 16916567, 21859206, 25186285, 26784347, 29218587, 29406610, 33741990]


[9309949, 12887631, 22419565, 31854264, 34512487]

### Task 1

Implement the function `and_merge` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already, and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [111]:
def and_merge(sorted_list1, sorted_list2):
    merged_list = []
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)

    #iterators
    it_1 = 0
    it_2 = 0

    while it_1 < len(list1) and it_2 < len(list2): #  checking if it exceeds the max len of any list, merge_list can have at most len(min(sorted_list1,sorted_list2)) elements
        if list1[it_1] == list2[it_2]:
            merged_list.append(list1[it_1])
            it_1 += 1
            it_2 += 1
        elif list1[it_1] < list2[it_2]:
            it_1 += 1
        else:
            it_2 += 1

    return merged_list

sampl1= [4,9309949, 12887631, 16916569, 22419565, 31854264, 34512487] 
sample2= [1,4,6,6,6,6,6,6,6,6,6,9756244, 16916567, 16916569, 21859206,22419565, 25186285, 26784347, 29218587, 29406610, 33741990]

# testing:
print(and_merge(sampl1,sample2))
print(and_merge(inverted_index['group'],inverted_index['team']))

[4, 16916569, 22419565]
[2513080, 2597883, 7149759, 9458505, 15058096, 16001596, 18954194, 19161959, 23204355, 23266169, 25455331, 29278692, 30356814, 30482104, 30707087, 30957656, 31419610, 31507492, 32435316, 33836214, 33955347, 34058956]


### Task 2

Similarly as above, implement the function `or_merge` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists. Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [112]:
def or_merge(sorted_list1, sorted_list2):
    merged_list = []
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)

    for id in list1: merged_list.append(id)

    for id in list2:
        if id in merged_list: continue
        
        merged_list.append(id)

    return merged_list

# testing:
print(or_merge(sampl1,sample2))

[4, 9309949, 12887631, 16916569, 22419565, 31854264, 34512487, 1, 6, 9756244, 16916567, 21859206, 25186285, 26784347, 29218587, 29406610, 33741990]


### Task 3

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `and_merge` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [113]:
def and_query(query, docs=[]):
	if type(query) == str:		#initial state where query is a string
		query = preprocess(tokenize(query))
		docs.extend(inverted_index[query[0]]) #fill docs with documnts that include only the first term to be able to compare with something later one in the recursion

	if len(query) <= 1:  #'bottom" of recursion where there's only one term left
		return and_merge(docs,inverted_index[query[0]])
	else:
		docs = and_merge(docs, inverted_index[query[0]]) #find common docs between already found document and documets including the next term - for the firt 'iteraion' docs and the term are the same documents

		return and_query(query[1:], docs) #run and_query again with all the query exluding the current first term


print(and_query('scientific emotion deal'))

[2238970, 6858287, 11478037, 24712897, 30534094, 30711434]


### Task 4

Construct another function called `or_query` that works in the same way as `and_query` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `or_merge` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [114]:
def or_query(query, docs=[]):
	if type(query) == str:		#initial state where query is a string
		query = preprocess(tokenize(query))
		docs.extend(inverted_index[query[0]]) #fill docs with documnts that include only the first term to be able to compare with something later one in the recursion

	if len(query) <= 1:  #'bottom" of recursion where there's only one term left
		return or_merge(docs,inverted_index[query[0]])	

	docs.extend(or_merge(docs, inverted_index[query[0]])) #find common docs between already found document and documets including the next term - for the firt 'iteraion' docs and the term are the same documents

	return or_query(query[1:], docs) #run and_query again with all the query exlusing the current first term

or_query('athens amsterdam')


[16938038,
 22862968,
 25797829,
 32853011,
 34177648,
 34554917,
 16938038,
 22862968,
 25797829,
 32853011,
 34177648,
 34554917,
 9756244,
 16916567,
 21859206,
 25186285,
 26784347,
 29218587,
 29406610,
 33741990]

### Task 5

Why does `and_query('emotional factor experiment')` not return our example paper 34648542, even though it mentions emotional factors and experiments in the abstract? (You do not have to implement anything to fix this yet!)

**Answer:** [Write your answer text here]

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.